## Create Agents to Research and Write an Article

##### !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [15]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [16]:
from crewai import Agent, Task, Crew

In [17]:
import os
from util import get_openai_api_key, get_openai_model_name

openai_api_key = get_openai_api_key()
openai_model_name = get_openai_model_name()

## Creating Agents
Define your Agents, and provide them a role, goal and backstory. It has been seen that LLMs perform better when they are role playing.



In [18]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article about the topic: {topic}."
              "You collect information that helps the audience learn something and make informed decisions.\n"
              "Your work is the basis for the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

In [19]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic. "
              "You follow the main objectives and direction of the outline, as provide by the Content Planner. "
              "You also provide objective and impartial insights and back them up with information provide by the Content Planner. "
              "You acknowledge in your opinion piece when your statements are opinions as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [20]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post from the Content Writer. "
              "Your goal is to review the blog post to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints when providing opinions or assertions, "
              "and also avoids major controversial topics or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks
Define your Tasks, and provide them a description, expected_output and agent.



### Task: Plan

In [21]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering their interests and pain points.\n"
        "3. Develop a detailed content outline including an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [22]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named in an engaging manner.\n"
        "4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [23]:
edit = Task(
    description="Proofread the given blog post for grammatical errors and alignment with the brand's voice.",
    expected_output="A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

Create your crew of Agents
Pass the tasks to be performed by those agents.

Note: For this simple example, the tasks will be performed sequentially (i.e they are dependent on each other), so the order of the task in the list matters.
verbose=True allows you to see all the logs of the execution.



In [27]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

In [28]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Comprehensive Content Plan for Blog Article on Artificial Intelligence**

**1. Latest Trends in Artificial Intelligence:**
   - **Generative AI:** Emphasis on advancements in generative AI models like OpenAI's GPT series, their applications in content creation, programming, and simulations.
   - **AI in Healthcare:** Innovations relating to AI in diagnosis, treatment planning, and personalized medicine.
   - **Edge AI:** Growth of AI processing at the edge, reducing dependency on cloud computing.
   - **Ethical AI and Regulations:** Increasing focus on developing ethical AI frameworks an

In [30]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# Navigating the Future: Trends and Implications of Artificial Intelligence

## Introduction

Artificial Intelligence (AI) has become a pivotal force in transforming the technological landscape of the 21st century. As we advance further into the digital age, AI's potential to revolutionize industries and everyday life remains unparalleled. In this blog, we delve into the current dynamics of AI, exploring the latest trends that are shaping its evolution and why they are significant to tech enthusiasts, industry professionals, and policymakers alike.

Amidst the plethora of developments, understanding AI's innovative strides and ethical considerations allows us to harness its capabilities while addressing pertinent societal concerns. So, why should we care? AI is not just a scientific curiosity; it is the backbone of impending technological transformations that hold promise and challenges in equal measure.

## Exploring the Latest Trends in AI

One of the most fascinating areas of development is **Generative AI**. Known for its ability to produce creative content, Generative AI models, such as OpenAI's GPT series, are redefining boundaries in content creation, programming, and simulations. These models can generate human-like text, art, and even design new molecules for drug discovery, showcasing a versatile blend of creativity and utility in various fields.

In the realm of healthcare, **AI innovations** are paving the way for breakthrough advancements. From diagnostics to treatment planning, AI is ushering in an era of personalized medicine. Innovations like AI algorithms that predict patient outcomes or assist radiologists are revolutionizing how we understand and treat health conditions, promising safer and more efficient healthcare ecosystems.

The rise of **Edge AI** highlights the shift towards processing data locally at the device level, reducing dependency on cloud computing. This evolution allows real-time data processing with increased privacy and reduced latency, ushering in a new wave of tech development where connected devices operate more intelligently and autonomously.

Furthermore, as AI becomes integral to the technological fabric, **Ethical AI** is gaining traction. Developing robust ethical frameworks and adopting emerging legislation to guide AI usage underline the importance of using these powerful tools responsibly. Ethical AI practices are essential to mitigate biases, ensure transparency, and protect privacy in an increasingly AI-driven world.

## Key Players in the AI Space

The arena of artificial intelligence is spearheaded by leading tech giants and research institutions like OpenAI, Google DeepMind, IBM Watson, Microsoft Azure AI, and NVIDIA. These companies are at the forefront of developing innovative AI solutions, propelling the industry towards new heights with each breakthrough.

Institutions such as the MIT-IBM Watson AI Lab, Stanford AI Lab, and the Allen Institute for AI also play a critical role in advancing AI research. Their collaborative efforts and initiatives pave the way for essential discoveries and technological innovations. Recently, significant announcements and partnerships from these players have showcased new AI models and applications that promise to further reshape the industry landscape.

## Impact of AI on Society and Industries

Artificial Intelligence is reshaping various sectors through comprehensive transformations. In industries such as manufacturing, AI-powered automation is optimizing production lines and enhancing quality control, demonstrating efficiency and cost-effectiveness. In retail, AI is reshaping consumer experiences through personalized shopping and inventory management.

AI is also redefining jobs and influencing organizational strategies. While some fear the specter of job displacement, AI encourages the creation of new roles centered around AI maintenance and enhancement. Companies are adjusting strategies to leverage AI for competitive advantage, thus influencing economic structures and consumer dynamics.

## Addressing Common Concerns in AI

Despite its promising potential, AI poses several concerns, notably in job displacement and ethical use. It is crucial to develop strategies that integrate AI into the workforce, reskilling and upskilling employees to coexist with AI technologies. Addressing these shared concerns can mitigate social risk while allowing AI to thrive responsibly.

AI also poses challenges in terms of privacy. The vast amounts of data processed by AI technologies necessitate stringent regulatory measures to safeguard user information. Ensuring robust privacy frameworks guided by emerging regulations is essential in addressing the pervasive concerns surrounding AI data usage.

## Conclusion and Call to Action

AI's transformative potential presents a dual role, offering unparalleled opportunities alongside significant challenges. By embracing ethical practices and remaining informed about AI advancements, individuals and organizations can harness AI's benefits while mitigating its risks. As we continue to navigate the rapidly evolving AI landscape, it remains imperative that we engage responsibly, ensuring that this powerful technology serves the greater good.

As we venture into the future, I encourage all stakeholders—whether tech enthusiasts, policymakers, or industry leaders—to engage actively with AI discussions, advocate for ethical practices, and remain aware of its exponential growth. Together, we can chart a responsible path forward, leveraging AI's incredible capabilities to build a better, more inclusive world.
```

```# Other Popular Models as LLM for your Agents

-----------------------------------------
## Hugging Face (HuggingFaceHub endpoint)
-----------------------------------------

from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function

--------------
## Mistral API
--------------

OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"

----------
## Cohere
----------

from langchain_community.chat_models import ChatCohere

# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"

llm = ChatCohere()

### you will pass "llm" to your agent function
```
